########## REQUIRE A API KEY FROM ALPHA VANTAGE (FREE)<br><br>
This script is the extract the historical stock prices from the Alpha Vantage API.<br>
Since the API for free users is limited, the definitions (in the last cell) will have to be ran one at a time,
one per day...<br><br>
Do note that in the last two lines, the dataframes are empty.



In [26]:
import pandas, json, numpy, requests, os, datetime, pytz, time, re

# 1. I want to obtain data on stocks, mainly focusing on historical data. ------------------------------------------
### To use API functions and automation using loops, I need to create a list of the ticks I want to obtain data from:

# I obtained the following list of ticks as a txt from http://www.eoddata.com/symbols.aspx from the New York Stock Exchange and the American Stock Exchange:
NYSE_csv = pandas.read_csv('NYSE.txt', sep="\t", header=0).set_index('Symbol')
NYSE_csv.to_csv('NYSE_csv.csv')

AMEX_csv = pandas.read_csv('AMEX.txt', sep="\t", header=0).set_index('Symbol')
AMEX_csv.to_csv('AMEX_csv.csv')

# The two csv files creates were only made to check the structure of the datasets.
# Merge the two DataFrames and ordering according to ticks alphabetically.
# Removing all ticks with "-" or "." because the data for those are not provided by the Alpha Vantage API.
# Removing these ticks will also prevent errors from occuring when requesting obtaining errors from API.
stock_exchange_ticks_and_names = pandas.merge(NYSE_csv.reset_index(), AMEX_csv.reset_index(), how='outer')
stock_exchange_ticks_and_names.to_csv('merged_NYSE_AMEX.csv')
stock_exchange_ticks_and_names_copy = stock_exchange_ticks_and_names.copy().dropna()

## Removing companies with special characters or the "Cl A" and "Cl B" = class A/B stocks
regex1 = re.compile('[@_!#$%^&*()<>?/\|}{~:[\].]')
regex2 = re.compile('Cl ')

stock_exchange_ticks_and_names_removed = pandas.DataFrame()

for x, y in stock_exchange_ticks_and_names_copy.iterrows():

    if bool(regex1.search(y['Description'])) == False and bool(regex2.search(y['Description'])) == False and bool(regex1.search(y['Symbol'])) == False:
        stock_exchange_ticks_and_names_removed.loc[x, 'Symbol'] = y['Symbol']
        stock_exchange_ticks_and_names_removed.loc[x, 'Description'] = y['Description']

stock_exchange_ticks_and_names_removed = stock_exchange_ticks_and_names_removed.set_index('Symbol')
stock_exchange_ticks_and_names_removed = stock_exchange_ticks_and_names_removed.reset_index()
stock_exchange_ticks_and_names_removed.to_csv('merged_NYSE_AMEX_removed_alpha.csv')


# Looping through Alpha Vantage API to obtain historical data for the last 20 years
## Due to restrictions on daily requests, only 500 requests to the API can be made
## per day and 5 requests per minute. I created smaller DataFrames out of 
## stock_exchange_ticks_and_names to lessen the time it takes to run the requests, as well as
## prevent the gathering repeats of the same ticks--should the requests get interrupted.
### This part of the code will create a new csv file for each tick to eliminate the need to 
### creating one large DataFrame with the data from all 148 ticks.
stock_exchange_ticks_and_names_0_399 = stock_exchange_ticks_and_names_removed.iloc[0:400, :]
stock_exchange_ticks_and_names_400_799 = stock_exchange_ticks_and_names_removed.iloc[400:800, :]
stock_exchange_ticks_and_names_800_1199 = stock_exchange_ticks_and_names_removed.iloc[800:1200, :]
stock_exchange_ticks_and_names_1200_1599 = stock_exchange_ticks_and_names_removed.iloc[1200:1600, :]
stock_exchange_ticks_and_names_1600_1999 = stock_exchange_ticks_and_names_removed.iloc[1600:2000, :]
stock_exchange_ticks_and_names_2000_2399 = stock_exchange_ticks_and_names_removed.iloc[2000:2400, :]
stock_exchange_ticks_and_names_2400_2799 = stock_exchange_ticks_and_names_removed.iloc[2400:2800, :]
stock_exchange_ticks_and_names_2800_3199 = stock_exchange_ticks_and_names_removed.iloc[2800:3200, :]
stock_exchange_ticks_and_names_3200_3599 = stock_exchange_ticks_and_names_removed.iloc[3200:3600, :]
stock_exchange_ticks_and_names_3600_3999 = stock_exchange_ticks_and_names_removed.iloc[3600:4000, :]
stock_exchange_ticks_and_names_4000_4399 = stock_exchange_ticks_and_names_removed.iloc[4000:4400, :]
stock_exchange_ticks_and_names_4400_4799 = stock_exchange_ticks_and_names_removed.iloc[4400:4800, :]
stock_exchange_ticks_and_names_4800_4880 = stock_exchange_ticks_and_names_removed.iloc[4800:, :]

stock_indices_df = pandas.DataFrame({'Description': ['S&P', 'Dow', "Nasdaq"], 'Symbol': ['.INX', '.DJI', ".IXIC"]})


def daily_stock_request(stock_ticks_names):
    filepath = os.getcwd() + '\\Daily Stock Prices\\'
    working_stocks = []

    for x, z in stock_ticks_names.iterrows():
        print(z['Symbol'])  ### This is to indicate where a error might have occurred.
        try:
            get_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=' + z['Symbol'] + '&outputsize=full&dataype=json&apikey=' ### INPUT API KEY
            alpha_vantage_api_requests = requests.get(get_URL)
            alpha_vantage_api_convert = alpha_vantage_api_requests.json()

            time.sleep(20)

            if 'Time Series (Daily)' in alpha_vantage_api_convert.keys():
                daily_stock_df = pandas.DataFrame()
                date_list = list(alpha_vantage_api_convert['Time Series (Daily)'].keys())

                for y, u in zip(date_list, range(len(date_list))):
                    daily_stock_df.loc[u, 'Symbol'] = z['Symbol']
                    daily_stock_df.loc[u, 'Name'] = z['Description']
                    daily_stock_df.loc[u, 'Date'] = y
                    daily_stock_df.loc[u, 'Open'] = alpha_vantage_api_convert['Time Series (Daily)'][y]['1. open']
                    daily_stock_df.loc[u, 'High'] = alpha_vantage_api_convert['Time Series (Daily)'][y]['2. high']
                    daily_stock_df.loc[u, 'Low'] = alpha_vantage_api_convert['Time Series (Daily)'][y]['3. low']
                    daily_stock_df.loc[u, 'Close'] = alpha_vantage_api_convert['Time Series (Daily)'][y]['4. close']
                    daily_stock_df.loc[u, 'Volume'] = alpha_vantage_api_convert['Time Series (Daily)'][y]['5. volume']
                print('8')
                
                if not os.path.exists(filepath):
                    os.makedirs(filepath)

                ### Updates the current .csv file with the new dates if it already exists.
                if os.path.exists(filepath + z['Symbol'] + ' - ' + z['Description'] + '.csv') == True:
                    temp_df = pandas.read_csv(filepath + z['Symbol'] + ' - ' + z['Description'] + '.csv')
                    print('4')
                    if temp_df.Date.max() < daily_stock_df.Date.max():
                        print('1')
                        difference_df = daily_stock_df[daily_stock_df.Date > temp_df.Date.max()]
                        print('2')
                        master_df = pandas.concat([temp_df, difference_df]).sort_values(by='Date', ascending=False)
                        print('3')
                        master_df.to_csv(filepath + z['Symbol'] + ' - ' + z['Description'] + '.csv')
                    working_stocks.append(z['Description'])
                    print(working_stocks)
                print('7')
                if os.path.exists(filepath + z['Symbol'] + ' - ' + z['Description'] + '.csv') == False:
                    print('6')
                    daily_stock_df.to_csv(filepath + z['Symbol'] + ' - ' + z['Description'] + '.csv')
                    working_stocks.append(z['Description'])
                    print(working_stocks)

        except Exception:
            pass

    return working_stocks

In [ ]:
daily_stock_request(stock_indices_df)  ## Run everytime; Nasdaq does not return any data even after different changes

working_stocks1 = daily_stock_request(stock_exchange_ticks_and_names_0_399)  # 390 stocks working
working_stocks2 = daily_stock_request(stock_exchange_ticks_and_names_400_799) # 399 stocks working
working_stocks3 = daily_stock_request(stock_exchange_ticks_and_names_800_1199) # 398 stocks working
working_stocks4 = daily_stock_request(stock_exchange_ticks_and_names_1200_1599) # 400 stocks working
working_stocks5 = daily_stock_request(stock_exchange_ticks_and_names_1600_1999) # 387 stocks working
working_stocks6 = daily_stock_request(stock_exchange_ticks_and_names_2000_2399) # 388  stocks working
working_stocks7 = daily_stock_request(stock_exchange_ticks_and_names_2400_2799) # 388 stocks working
working_stocks8 = daily_stock_request(stock_exchange_ticks_and_names_2800_3199) # 395 stocks working
working_stocks9 = daily_stock_request(stock_exchange_ticks_and_names_3200_3599) # 398 stocks working
working_stocks10 = daily_stock_request(stock_exchange_ticks_and_names_3600_3999) # 396 stocks working
working_stocks11 = daily_stock_request(stock_exchange_ticks_and_names_4000_4399) # 81 stocks working (only 81 stocks in df)
working_stocks12 = daily_stock_request(stock_exchange_ticks_and_names_4400_4799) # no stocks in df
working_stocks13 = daily_stock_request(stock_exchange_ticks_and_names_4800_4880) # no stocks in df
